# Evolutionary Computation - Assignment 4: Candidate moves

Bartosz Stachowiak 148259<br>
Andrzej Kajdasz 148273

## 1. Problem Statement

There are columns of integers representing nodes. Each row corresponds to a node and contains its x and y coordinates in a plane, as well as a cost associated with the node. There were 4 such data sets each consisting of 200 rows (each representing a single node).

Problem to solve is to choose precisely 50% of the nodes (rounding up if there is an odd number of nodes) and create a Hamiltonian cycle (a closed path) using this subset of nodes. The goal is to minimize the combined total length of the path and the total cost of the selected nodes.

To calculate the distances between nodes, the Euclidean distance formula was used and then round the results to the nearest integer. As suggested, the distances between the nodes were calculated after loading the data and placed in a matrix, so that during the subsequent evaluation of the problem, it was only necessary to read these values which reduced the cost of the operation of the algorithm.

To solve the problem the local search algorithm (steepest) with candidate moves was used. We operate in edge-intra and inter neighbourhood.

## 2. Pseudocode of all implemented algorithms

### Local Steepest Search with Candidates

```
function generate_candidate_operations(solution, nodes, distance_matrix, closest_nodes):
    UNVISITED = -1
    node_solution_index = [UNVISITED] * len(nodes)
    for i in range(len(solution)):
        node_solution_index[solution[i]] = i
    
    operations = []
    for i in range(len(solution)):
        i_successor = (i + 1) % len(solution)
        i_predecessor = (i - 1) % len(solution)
        for j in range(len(closest_nodes[i])):
            candidate = node_solution_index[closest_nodes[i][j]]
            candidate_solution_index = node_solution_index[candidate]
            
            candidate_not_in_solution = candidate_solution_index == UNVISITED

            if candidate_not_in_solution:
                operations.append(NodeReplaceOperation(i_successor, candidate))
                operations.append(NodeReplaceOperation(i_predecessor, candidate))
            else:
                candidate_predecessor = (candidate_solution_index - 1) % len(solution)
                operations.append(EdgeSwapOperation(i, candidate_solution_index))
                operations.append(EdgeSwapOperation(i_predecessor, candidate_predecessor))

    return operations

``` 


```
function local_steepest_candidate_search(solution, nodes, distance_matrix, num_candidates):
    closest_nodes = index_closest_nodes(nodes, distance_matrix, num_candidates)    
    while true:
        operations = generate_candidate_operations(solution, nodes, distance_matrix, closest_nodes)

        best_operation = None
        best_delta = 0
        for operation in operations:
            delta = op.evaluate_delta(solution, nodes, distance_matrix)
            if delta < best_delta:
                best_operation = operation
                best_delta = delta

        if best_operation is None:
            break
        solution = selected_operation.apply(solution)

    return solution
```

## 3. Results of the computational experiments

### 3.1. Code for visualization of the results

In [ ]:
import pathlib
import itertools

import numpy as np
import matplotlib.pyplot as plt

import pandas as pd
from common import *

In [ ]:
DATA_FOLDER = '../data/'
OLD_RESULTS_FOLDER = f'{DATA_FOLDER}old_results/'
RESULT_FOLDER = f'{DATA_FOLDER}results/'
INSTANCE_FOLDER = f'{DATA_FOLDER}tsp_instances/'

SOLVERS = {
    "lsc-5-r" : "Steepest LS, with (5) Candidates",
    "lsc-10-r" : "Steepest LS, with (10) Candidates",
    "lsc-15-r" : "Steepest LS, with (15) Candidates",
    "lsc-20-r" : "Steepest LS, with (20) Candidates",
    "lsc-50-r" : "Steepest LS, with (50) Candidates",
}

OLD_SOLVERS = {
    'n': "Nearest Neighbor",
    'g': "Greedy Cycle",
    'd-1': "Greedy 2-regret heuristics",
    'd-1.25': "Weighted greedy heuristics (0.25)",
    'd-1.75': "Weighted greedy heuristics (0.75)",
    'r': "Random",
    'd': "Greedy Heuristic",
    'lssnode-r' : "Steepest LS, node neighborhood (random)",
    'lssedge-r' : "Steepest LS, edge neighborhood (random)",
    'lssboth-r' : "Steepest LS, both neighborhood (random)",
    'lsgnode-r' : "Greedy LS, node neighborhood (random)",
    'lsgedge-r' : "Greedy LS, edge neighborhood (random)",
    'lsgboth-r' : "Greedy LS, both neighborhood (random)",
    'lssnode-d' : "Steepest LS, node neighborhood (GH)",
    'lssedge-d' : "Steepest LS, edge neighborhood (GH)",
    'lssboth-d' : "Steepest LS, both neighborhood (GH)",
    'lsgnode-d' : "Greedy LS, node neighborhood (GH)",
    'lsgedge-d' : "Greedy LS, edge neighborhood (GH)",
    'lsgboth-d' : "Greedy LS, both neighborhood (GH)",
}
SOLVERS_TO_PLOT = SOLVERS.copy()
SOLVERS_TO_PLOT.update({"lssedge-r": OLD_SOLVERS["lssedge-r"]})
SOLVERS_NO_TIME = ["n","g","d-1","d-1.75","d-1.25"]
SOLVERS.update(OLD_SOLVERS)
NUM_NODES = 200

instance_files = [path for path in pathlib.Path(INSTANCE_FOLDER).iterdir() if path.is_file()]
instance_names = [path.name[:4] for path in instance_files]

In [ ]:
instances_data = {
    name: read_instance(f'{INSTANCE_FOLDER}{name}.csv')
    for name in instance_names
}

In [ ]:
instances_solvers_pairs = itertools.product(instances_data.keys(), SOLVERS.keys())

all_results = {}
all_costs = {}
all_times = {}
all_stats = {}

for instance, solver in instances_solvers_pairs:
    all_results[instance] = all_results.get(instance, {})
    all_costs[instance] = all_costs.get(instance, {})
    all_times[instance] = all_times.get(instance, {})
    all_stats[instance] = all_stats.get(instance, {})
    costs = []
    times = []
    paring_results = []
    for idx in range(NUM_NODES):
        if(solver in SOLVERS_NO_TIME):
            time = 0
            solution, cost = read_solution_timeless(f'{OLD_RESULTS_FOLDER}{instance}-{solver}-{idx}.txt')
        elif(solver in OLD_SOLVERS):
            solution, cost, time = read_solution(f'{OLD_RESULTS_FOLDER}{instance}-{solver}-{idx}.txt')
        else:
            solution, cost, time = read_solution(f'{RESULT_FOLDER}{instance}-{solver}-{idx}.txt')
        paring_results.append(solution)
        costs.append(cost)
        times.append(time)
    all_results[instance][solver] = np.array(paring_results)
    all_costs[instance][solver] = np.array(costs)
    all_stats[instance][solver] = {
        'mean': np.mean(costs),
        'std': np.std(costs),
        'min': np.min(costs),
        'max': np.max(costs),
    }
    all_times[instance][solver] = {
        'mean': np.mean(times),
        'std': np.std(times),
        'min': np.min(times),
        'max': np.max(times),
    }

In [ ]:
costs_df = pd.DataFrame(all_stats).T
time_df = pd.DataFrame(all_times).T
max_df = pd.DataFrame(all_stats).T
min_df = pd.DataFrame(all_stats).T
mean_time_df = pd.DataFrame(all_times).T

for column in SOLVERS.keys():
    costs_df[column] = costs_df[column].apply(lambda x: f'{x["mean"]:.0f} ({x["min"]:.0f} - {x["max"]:.0f})')
    time_df[column] = time_df[column].apply(lambda x: f'{x["mean"]/1000:.2f} ({x["min"]/1000:.2f} - {x["max"]/1000:.2f})')
    max_df[column] = max_df[column].apply(lambda x: x['max'])
    min_df[column] = min_df[column].apply(lambda x: x['min'])
    mean_time_df[column] = mean_time_df[column].apply(lambda x: x['mean']/1000)

for df in [costs_df, time_df, max_df, min_df, mean_time_df]:
    df.rename(columns=SOLVERS, inplace=True)
time_df = time_df.filter(items = SOLVERS_TO_PLOT.values())
mean_time_df = mean_time_df.filter(items  = SOLVERS_TO_PLOT.values())

### 3.2. Visualizations and statistics of cost for all dataset-algorithm pairs

In tabular form we present the Mean, Minimum and Maximum of the results of the algorithms for each dataset.

In [ ]:
print("Mean (min-max) of the costs:")

best_means = {
    instance: min(all_stats[instance][solver]['mean'] for solver in SOLVERS.keys())
    for instance in instance_names
}

def apply_style(v: str, best_val: float):
    num = v.split()[0]
    try:
        num = float(num)
    except ValueError:
        return ""
    if round(num) == round(best_val):
        return "font-weight: bold; color: red"
    return ""
    


costs_df.T.style.apply(lambda x: [
    apply_style(v, best_means[x.index[i]])
    for i, v in enumerate(x)
], axis = 1)

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(15, 8), sharey=True)

for idx, instance in enumerate(instances_data.keys()):
    if idx%2 == 0:
        axs[(idx//2)%2][idx%2].set_ylabel('Cost')
    axs[(idx//2)%2][idx%2].set_title(instance)

    axs[(idx//2)%2][idx%2].violinplot(
        [all_costs[instance][solver] for solver in SOLVERS_TO_PLOT.keys()],
        showmeans=True,
    )

    axs[(idx//2)%2][idx%2].set_xticks(range(1, len(SOLVERS_TO_PLOT.keys()) + 1))
    if idx > 1:
        axs[(idx//2)%2][idx%2].set_xticklabels(SOLVERS_TO_PLOT.values(), rotation=45, ha='right')
    else :
        axs[(idx//2)%2][idx%2].set_xticklabels([])

plt.suptitle('Distribution of the costs')
plt.show()

 ### 3.3. Visualizations and statistics of running times for all dataset-algorithm pairs

In [ ]:
print("Mean (min-max) of the time [ms]:")

best_times = {
    instance: min(all_times[instance][solver]['mean'] for solver in SOLVERS_TO_PLOT.keys()) / 1000
    for instance in instance_names
}

def apply_style(v: str, best_val: float):
    num = v.split()[0]
    try:
        num = float(num)
    except ValueError:
        return ""
    if round(num) == round(best_val):
        return "font-weight: bold; color: red"
    return ""
    


time_df.T.style.apply(lambda x: [
    apply_style(v, best_times[x.index[i]])
    for i, v in enumerate(x)
], axis = 1)

In [ ]:
x_range = np.arange(len(SOLVERS_TO_PLOT))
bar_width = 0.8 / len(instances_data.keys())
mean_time_plot_df = mean_time_df.T.sort_values(by="TSPA", ascending=False).T
fig, ax = plt.subplots(figsize=(15, 8), sharey=True)
for idx, instance in enumerate(instances_data.keys()):
     ax.bar(
          x_range + idx * bar_width,
          height=mean_time_plot_df.loc[instance].values,
          width=bar_width,
          label=instance,
     )
ax.set_xticks(x_range + bar_width * (len(instances_data.keys()) - 1) / 2)
ax.set_xticklabels(mean_time_plot_df.columns, rotation=45, ha='right')
plt.title('Time per instance per solver')
plt.ylabel('Running Time [ms]')
plt.legend()
plt.show()


In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(14, 5))
N = [5, 10, 15, 20, 50]


for instance in instances_data.keys():
    
    axs[0].plot(
        N,
        [all_stats[instance][f"lsc-{n}-r"]['mean'] for n in N],
        label=instance,
    )
    axs[1].plot(
        N,
        [all_times[instance][f"lsc-{n}-r"]['mean']/1_000 for n in N],
        label=instance,
    )

axs[0].set_title('Mean cost of Steapest LS with different number of candidates')
axs[0].set_xlabel('Number of candidates')
axs[0].set_ylabel('Mean cost')

axs[1].set_title('Mean time of Steapest LS with different number of candidates')
axs[1].set_xlabel('Number of candidates')
axs[1].set_ylabel('Mean time [ms]')

plt.legend()
plt.show()

## 4. Best solutions for all datasets and algorithms

To more easily compare the results, we present the best solutions for each dataset side by side.

The weight of each node is denoted both by its size and color. The bigger and brighter the node, the higher its weight.

### 4.1 New algortithms

In [ ]:
for solver_idx, solver in enumerate(SOLVERS_TO_PLOT.keys()):
    fig, axs = plt.subplots(1, 4, figsize=(20, 5))
    for idx, instance in enumerate(instances_data.keys()):
        best_instance_idx = np.argmin(all_costs[instance][solver])
        plot_solution_for_instance(instances_data[instance], all_results[instance][solver][best_instance_idx], axs[idx])
        axs[idx].set_title(f'{instance}: {all_costs[instance][solver][best_instance_idx]:.0f}')
    fig.suptitle(f'{SOLVERS_TO_PLOT[solver]}', fontsize=16, y=1.05)
plt.show()

### 4.2 Best solution for each instance from all algorithms

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(20, 5))
for idx, instance in enumerate(instances_data.keys()):
    best_cost =  np.inf
    for solver_idx, solver in enumerate(SOLVERS.keys()):
         if best_cost > np.min(all_costs[instance][solver]):
                best_cost = np.min(all_costs[instance][solver])
                best_result = all_results[instance][solver][np.argmin(all_costs[instance][solver])], 
                best_solver = solver
    best_instance_idx = np.argmin(all_costs[instance][best_solver])
    plot_solution_for_instance(instances_data[instance], all_results[instance][best_solver][best_instance_idx], axs[idx])
    axs[idx].set_title(f'{instance}: {all_costs[instance][best_solver][best_instance_idx]:.0f}')
    print(instance)
    print(f'\tSolver: {SOLVERS[best_solver]}, Total cost: {best_cost}')
    zero_index = np.where(best_result[0] == 0)[0][0]
    nodes = list(best_result[0][zero_index:])+list(best_result[0][:zero_index])
    print(f'\t Nodes: {nodes}\n')
plt.show()

## 5. Source Code

[GitHub](https://github.com/Tremirre/ECP)

## 6. Conclusions



Analyzing the results and visualizations, one can come to several conclusions about the algorithms used in the task:
- **Time of running local search algorithm (steepest) with candidate is greatly reduced** in comparison with its version without candidates. This is due to the fact that the number of iterations of the algorithm is reduced, because the algorithm does not have to check all possible neighbours, but only a few of them.
- The running time for steepest local search with *N* candidates grows linearly with *N*. This is due to the fact that the algorithm has to check all *N* candidates, and makes *N* * *|S|* * 2 comparisons per interation, where *|S|* - solution size (constant), so the total time is linearly proportional to *N* (even though the number of iterations may vary).
- The mean cost of the solution found by the algorithm with candidates decreases as the number of candidates *N* increases in a non-linear way - initial decrease is very steep, and then it slows down. Any number of candidates greated than 15 does not seem to have a significant impact on the minimum cost of the solution found by the algorithm, whilst the time of running the algorithm increases significantly.
- Experiments with the number of candidates also yielded some interesting findings: 
    - The number of 5 candidates is too low, i.e. it shows a significant deterioration of results relative to the version with more candidates.  
    - The best results can be seen for the number of candidates between 10 and 20. Increasing this number to 50 can result in worse results (TSPD, TSPC, min in TSPB). 
- The steepest local search algorithm without candidates seems to achieve similar results for its candidate counterparts with *N* larger than 10, although it's worth noting that - counterintuitively - the algorithm with candidates (ergo - with restricted neighborhood) can achieve better results in terms of mean cost:
    - LSC50 for TSPA (77845 < 77856)
    - LSC50 for TSPB (71259 < 71288)
    - LSC15 for TSPC (51370 < 51427)
    - LSC20 for TSPD (48221 < 48253)